### 以下内容得到各订单对应商品的原价及乘以购买数量后的价格
输出文件: sku_sales_with_original_amount.csv

In [ ]:
# 读取订单数据及促销数据
import pandas as pd
sku_sales = pd.read_csv("../data/sku_sales.csv")
sku_price_and_status = pd.read_csv("../data/sku_price_and_status.csv")

In [ ]:
# 时间戳格式转换
sku_sales = sku_sales.rename(columns={'order_time':'date'})
sku_sales['date'] = pd.to_datetime(sku_sales['date']).dt.strftime('%Y-%m-%d')

In [ ]:
# 合并两表，得到各订单对应商品的原价格
sku_sales_with_price = pd.merge(sku_sales[['order_id', 'date', 'store_id', 'sku_id', 'quantity', 'channel']], 
                                  sku_price_and_status[['date', 'store_id', 'sku_id', 'original_price']], 
                                  on=['date', 'store_id', 'sku_id'], how='left')

In [ ]:
# 各订单对应商品原价格*购买件数=各订单购买商品总价(未计算折扣)
sku_sales_with_price['original_amount'] = sku_sales_with_price['quantity'] * sku_sales_with_price['original_price']

In [ ]:
# 排序
sku_sales_with_price.sort_values(by=['order_id', 'sku_id', 'channel'], ascending=[True, True, True, True, True], inplace=True)
sku_sales_with_price = sku_sales_with_price[['order_id', 'date', 'store_id', 'sku_id', 'channel', 'quantity', 'original_price', 'original_amount']]

In [ ]:
# 输出
sku_sales_with_price.to_csv("sku_sales_with_original_amount.csv", index = False)

---

### 以下内容得到每一笔订单中每一商品促销后的价格(算不动)

In [ ]:
import pandas as pd
df1 = pd.read_csv("./sku_sales_with_order_amount.csv")
df2 = pd.read_csv("./data/sku_prom.csv")

In [ ]:
# 表3为order_amounts 旨在记录每个订单的总金额，以供后续与threshold比对，计算折扣后金额
order_amounts = df1.groupby(['order_id', 'date', 'store_id', 'channel'])['original_amount'].sum().reset_index()

In [ ]:
# 给df1后每一行最后加上当前商品打折后的价格
def calculate_discounted_amount(row):
    order_id = row['order_id']
    date = row['date']
    store_id = row['store_id']
    channel = row['channel']
    order_amount = order_amounts[(order_amounts['order_id'] == order_id) & (order_amounts['date'] == date) & (order_amounts['store_id'] == store_id) & (order_amounts['channel'] == channel)]['original_amount'].values[0]
    sku_id = row['sku_id']
    promo_info = df2[(df2['store_id'] == store_id) & (df2['sku_id'] == sku_id) & (df2['date'] == date) & (df2['channel'] == channel)]
    if not promo_info.empty:
        threshold = promo_info['threshold'].values[0]
        discount_off = promo_info['discount_off'].values[0]
        original_amount = row['original_amount']
        if order_amount > threshold * original_amount:
            return original_amount * (1 - discount_off)
    
    return row['original_amount']  # 没有促销信息或不满足条件，原价
    
df1['discounted_amount'] = df1.apply(calculate_discounted_amount, axis=1)
# 然后再对df1作下一步处理？